In [ ]:
# ==========================================================================================================
# PROBLEM A4
#
# Build and train a binary classifier for the IMDB review dataset.
# The classifier should have a final layer with 1 neuron activated by sigmoid.
# Do not use lambda layers in your model.
#
# The dataset used in this problem is originally published in http://ai.stanford.edu/~amaas/data/sentiment/
#
# Desired accuracy and validation_accuracy > 83%
# ===========================================================================================================

In [64]:
#Check for GPU availability
import tensorflow as tf
print(tf.test.is_gpu_available())

#Check for python version
import sys
print(sys.version)

True
3.9.18 (main, Sep 11 2023, 13:41:44) 
[GCC 11.2.0]


2024-02-15 22:55:22.458749: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-15 22:55:22.459045: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-15 22:55:22.459089: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-15 22:55:22.459904: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-15 22:55:22.459918: I tensorflow/core/co

In [1]:
#Libraries
import tensorflow_datasets as tfds
import numpy as np
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

/root/miniconda3/envs/conda3.9/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-02-15 21:54:37.148956: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-15 21:54:38.052925: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [86]:
#The callback to reach the objective
class thecallbacks(tf.keras.callbacks.Callback):
    def __init__(self):
        super(thecallbacks, self).__init__()
    
    def on_epoch_end(self, epochs, logs=None):
        logtrain, logval = logs["accuracy"], logs["val_accuracy"]
        if logtrain > 0.83 and logval > 0.83:
            self.model.stop_training = True

In [66]:
#Const
AUTOTUNE = tf.data.AUTOTUNE
BATCH_SIZE = 32
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = "<OOV>"

In [67]:
#Load dataset from "imdb_reviews"
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

targets=info.features["label"].names        #Get targets
print(targets)

info.splits                                 #Check for n of both train and test

['neg', 'pos']


{'train': <SplitInfo num_examples=25000, num_shards=1>,
 'test': <SplitInfo num_examples=25000, num_shards=1>,
 'unsupervised': <SplitInfo num_examples=50000, num_shards=1>}

In [68]:
#seperate train_data, train_target, test_data and test_label

train_data = imdb["train"]
test_data = imdb["test"]

training_sentences, training_labels = [], []
for s, l in train_data:
    training_sentences.append(s.numpy().decode('utf8'))
    training_labels.append(l.numpy())

testing_sentences, testing_labels = [], []
for s, l in test_data:
    testing_sentences.append(s.numpy().decode('utf8'))
    testing_labels.append(l.numpy())

In [69]:
#The tokenizer
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)                      #Fit the tokenizer with the training_sentences

In [ ]:
#See the words saved on the tokenizer and the occurences
tokenizer.word_index

In [71]:
def get_padsequences(sentences):
    #Get how frequents the words are on the sentences, it's sorted by the first word that takes place in the sentences.
    sequences = tokenizer.texts_to_sequences(sentences)
    #This will return the list above to be matrices of how frequent the words are.
    return pad_sequences(sequences, truncating=trunc_type, maxlen=max_length)

training_padded = get_padsequences(training_sentences)
testing_padded = get_padsequences(testing_sentences)

In [74]:
def get_model():
    model=keras.Sequential([
        keras.layers.Embedding(input_dim=vocab_size, input_length=max_length, output_dim=embedding_dim),
        keras.layers.GlobalAvgPool1D(),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(32, activation="relu"),
        keras.layers.Dropout(0.3),        
        keras.layers.Dense(1, activation='sigmoid')
    ])   
    model.compile(optimizer=keras.optimizers.Adam(), loss=keras.losses.BinaryCrossentropy(), metrics="accuracy")
    return model

In [87]:
training_labels = np.array(training_labels)
testing_labels = np.array(testing_labels)

#changing data and label into a batched dataset adjusted performance wise.
train_ds = tf.data.Dataset.from_tensor_slices((training_padded, training_labels)).cache().batch(batch_size=BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)
test_ds = tf.data.Dataset.from_tensor_slices((testing_padded, testing_labels)).cache().batch(batch_size=BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)

model=get_model()
model.fit(train_ds, validation_data=test_ds, epochs=10, callbacks=[thecallbacks()])

Epoch 1/10
782/782 [==============================] - 35s 43ms/step - loss: 0.5340 - accuracy: 0.7424 - val_loss: 0.3872 - val_accuracy: 0.8285
Epoch 2/10
782/782 [==============================] - 8s 11ms/step - loss: 0.3265 - accuracy: 0.8640 - val_loss: 0.3675 - val_accuracy: 0.8373


# Answer!

In [92]:
# ==========================================================================================================
# PROBLEM A4
#
# Build and train a binary classifier for the IMDB review dataset.
# The classifier should have a final layer with 1 neuron activated by sigmoid.
# Do not use lambda layers in your model.
#
# The dataset used in this problem is originally published in http://ai.stanford.edu/~amaas/data/sentiment/
#
# Desired accuracy and validation_accuracy > 83%
# ===========================================================================================================

import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


def solution_A4():
    
    # Download dataset
    imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)
    
    # Get sentences and labels from both dataset
    train_data = imdb["train"]
    test_data = imdb["test"]
    training_sentences, training_labels = [], []
    for s, l in train_data:
        training_sentences.append(s.numpy().decode('utf8'))
        training_labels.append(l.numpy())
    testing_sentences, testing_labels = [], []
    for s, l in test_data:
        testing_sentences.append(s.numpy().decode('utf8'))
        testing_labels.append(l.numpy())
        
    #const
    BATCH_SIZE = 32
    AUTOTUNE = tf.data.AUTOTUNE
    vocab_size = 10000
    embedding_dim = 16
    max_length = 120
    trunc_type = 'post'
    oov_tok = "<OOV>"

    # Fit your tokenizer with training data
    tokenizer =  Tokenizer(oov_token=oov_tok, num_words=vocab_size)
    tokenizer.fit_on_texts(training_sentences)
    
    # Get pad sequences of each sentences
    def get_padsequences(sentences):
        ## Get how frequents the words are on the sentences, it's sorted by the first word that takes place in the sentences.
        sequences = tokenizer.texts_to_sequences(sentences)
        ## This will return the list above to be matrices of how frequent the words are.
        return pad_sequences(sequences, truncating=trunc_type, maxlen=max_length)
    
    training_padded = get_padsequences(training_sentences)
    testing_padded = get_padsequences(testing_sentences)
    
    # Change label type to numpy array, if not. It'll crash
    training_labels = np.array(training_labels)
    testing_labels = np.array(testing_labels)
    
    # Change to a ds and adjust it for performance
    def convert_to_ds(padded, labels):
        ds = tf.data.Dataset.from_tensor_slices((padded, labels))
        ds = ds.cache()
        ds = ds.batch(batch_size=BATCH_SIZE)
        ds = ds.prefetch(buffer_size=AUTOTUNE)
        return ds
    train_ds = convert_to_ds(training_padded, training_labels)
    test_ds = convert_to_ds(testing_padded, testing_labels)
    
    # The callback to stop upon reaching the objective
    class thecallbacks(tf.keras.callbacks.Callback):
        def __init__(self):
            super(thecallbacks, self).__init__()
        
        def on_epoch_end(self, epochs, logs=None):
            logtrain, logval = logs["accuracy"], logs["val_accuracy"]
            if logtrain > 0.83 and logval > 0.83:
                self.model.stop_training = True
            
    def get_model():
        model=keras.Sequential([
            keras.layers.Embedding(input_dim=vocab_size, input_length=max_length, output_dim=embedding_dim),
            keras.layers.GlobalAvgPool1D(),
            keras.layers.Dropout(0.3),
            keras.layers.Dense(32, activation="relu"),
            keras.layers.Dropout(0.3),        
            keras.layers.Dense(1, activation='sigmoid')
        ])   
        model.compile(optimizer=keras.optimizers.Adam(), loss=keras.losses.BinaryCrossentropy(), metrics="accuracy")
        return model
    
    model=get_model()
    model.fit(train_ds, validation_data=test_ds, epochs=10, callbacks=[thecallbacks()])
    
    return model


# The code below is to save your model as a .h5 file.
# It will be saved automatically in your Submission folder.
if __name__ == '__main__':
    # DO NOT CHANGE THIS CODE
    model = solution_A4()
    model.save("model_A4.h5")

Epoch 1/10
782/782 [==============================] - 53s 62ms/step - loss: 0.5163 - accuracy: 0.7583 - val_loss: 0.3835 - val_accuracy: 0.8288
Epoch 2/10
782/782 [==============================] - 10s 13ms/step - loss: 0.3198 - accuracy: 0.8679 - val_loss: 0.3689 - val_accuracy: 0.8373


/root/miniconda3/envs/conda3.9/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
